In [2]:
#from github.com/areed1192/td-ameritrade-python-api

import requests

from td_ameritrade_python_api import *
import re
from datetime import datetime as dt
from datetime import date as date
import pandas as pd
from pandas.tseries.offsets import BDay

import json



from td.utils import *

In [3]:
dt.now()

datetime.datetime(2021, 3, 17, 10, 58, 36, 53859)

In [4]:
def milliseconds_since_epoch(date):
#    return int(date.timestamp()*1000)
    return int(date.timestamp())*1000

In [5]:
milliseconds_since_epoch(dt.now())

1616003920000

In [6]:
# Import the client
from td.client import TDClient

with open('/home/cc/tdcred.json',"r") as read_file:
    cred = json.load(read_file)
    
#print(cred)

# Create a new session, credentials path is optional.
TDSession = TDClient(
    account_number=cred['accountID'],
    client_id = cred['client_id'],
    credentials_path = '/home/cc/Credentials/td_state.json',
    redirect_uri = "https://127.0.0.1"
)

# Login to the session
TDSession.login()

# Grab real-time quotes for 'MSFT' (Microsoft)
msft_quotes = TDSession.get_quotes(instruments='MSFT')

# Grab real-time quotes for 'AMZN' (Amazon) and 'SQ' (Square)
multiple_quotes = TDSession.get_quotes(instruments=['AMZN','SQ'])

Grabbing new access token...


In [7]:
TDSession.state['access_token']

'a/aBoyyDKBXOBWNfN0mqSfoiaofWHXO8q7IPe4RqoULn3VoujV/SxgH4VVTDT02U0lTSRaxilkalwS85gN84uIN8RZoXcn2KPnk6Z9+w6maK23rcmUteRAeU+EmsJD5OKaykJu8TtgkkOSmgc5+D8xpdqIkT7+CBMobFmmoBkPyLGJb68PCvQotjZupBe1L+AZS+ct8bwAP36hdFh1qjZzmemY8yWn6GDQcS5q1w3x19x4m+O+lABSXiJTDHwkzu1FNOIjl44Z/NN1LIjZnsr/HOWumySIgrNHWkA/fovXNMrwfyMuWAAtStADJK1J2TSRylJ74klnX+Xaz80er5cgKQHdFsiGXNWBj9DJzf0mpxp6i0358sZlIkPU3pRSJtnxyZ2ASQdu0/f0rTwkhINBBtjcXSVjqTmCTwCeou/wbFqfH/YRacarAuhw/o5cdFPoss52Tn5rzs+zF6OQW/bsnb8RrB5LN2EunG0a1/TznMmjrCV/ihHYUQAilmAuMyxlq6RplQd+de70/gvLxGuWvkuFv3OdS3YehZbqi9xM6hHwbMqPIl7fDtpK2100MQuG4LYrgoVi/JHHvlWlzxrGityGlAVUzXS1Ma8Z0aIhxKSGJRCgp4uS7JZrAGJ65o+okQgHShfAOZAt240QUvmb0XuHyHk1goc6PZ8+tH9Pl0sxA+1N9kgrlEi054/AudTRj62SUI76VqcnYImVhIncnN2sbikPv8DdkLPE5afs2TtCJhDsa+1VRTz52rYzAb3IXRLMjh+3IYuHn57sk4zWZ98rLOyUw07mjLmIrsmX5E655ZOtPlC6hG25Iolq/DwSf6gOKJrx3PDAlgdNrJc33cUtV6waJLXrmZh1MEVrgU5/Ix0uTqdLsXpAG8GXScLCXvKgLr2HpbYHEafGNXzQhpgjan7/xPSli7cXjwfEEiosFCLMMzf3luVJIsqrth8cM7YBtToLWmB44/o4WSTs8IJrk4Nj/wqKwDGRc

In [8]:
def extract_expiration_strike(symbol):
    date_type_strike = re.split('_',symbol)
    date_strike = re.split(r'[P,C]',date_type_strike[1])
    
    expiration = dt.strptime(date_strike[0],'%m%d%y').date()
    
    return expiration,float(date_strike[1])

    
    

In [9]:
#multiple_quotes

In [ ]:
positions = TDSession.get_accounts(fields = ['positions'])

if positions == None:
    print("Can't get positions")


In [ ]:


def show_positions(positions):
    
    for account in positions:
        
        if 'securitiesAccount' not in account:
            continue
            
        securitiesAccount = account['securitiesAccount']
        
        print("Account ID: ",securitiesAccount['accountId'])
        
        if 'positions' not in securitiesAccount.keys():
            print("no positions")
            continue

        for p in securitiesAccount['positions']:
    
            pdf = pd.DataFrame(p)
    
            i = p['instrument']
    
            if i['assetType'] == 'EQUITY':
                print(i['symbol'])
    
            if i['assetType'] == 'OPTION':
        
                expiration,strike = extract_expiration_strike(i['symbol'])
    
                today = date.today()
    
                if today <= expiration:
        
                    average_price = p['averagePrice']
                    netQuantity = p['settledLongQuantity'] + p['settledShortQuantity']
                    trade_value = average_price * 100 * netQuantity
                    market_value = p['marketValue']
        
                    pnl_since_open = market_value - trade_value


                    print(
                        i['underlyingSymbol'],
#            p['settledShortQuantity'],
#              p['settledLongQuantity'],
                          netQuantity,
#              i['underlyingSymbol'],
#              i['description'],
                          strike,
                          p['averagePrice'], # optionPrice per share
                          p['marketValue'],  # Quantity * 100 * optionPrice
                          pnl_since_open,
                          p['currentDayProfitLoss'])  # P/L Day

    
    
#        print(pdf)
#        print()
        
#        break

show_positions(positions)
        


In [ ]:
transactions = TDSession.get_transactions(account = cred['accountID'],transaction_type = 'TRADE')

In [ ]:
import pandas_market_calendars as mcal

nyse = mcal.get_calendar('NYSE')
#print(mcal.get_calendar_names())

def is_market_day(date):
    if (not len(pd.bdate_range(date,date))):
        print("it's a not a weekday")
        return False  # its not a weekday
    
    if nyse.valid_days(start_date = date,end_date = date) == 0:
        return False  # no valid market days in date range
    
    return True  # a weekday and not a holiday

def nearest_market_day(date):
    
#    print("nearest_market_day: ", type(date))

    if is_market_day(date):
        return date
    
#    print("going to previous day")
    
    return nearest_market_day(date - pd.Timedelta(days=1))

In [ ]:
def get_closing_price(expiration_date,symbol):
    
    #print("get_closing_price:",expiration_date,symbol)
#    print("get_closing_price:",symbol,type(expiration_date),expiration_date.date())
        
    expiration_date = nearest_market_day(expiration_date)
    
    expiration_date = expiration_date.replace(hour = 6,minute = 30, second = 0)
    
    print("get_closing_price nearest:",expiration_date,symbol)
    
    #print('get_closing_price: ',type(expiration_date),expiration_date.to_pydatetime())
    
#    print("get_closing_price since epoch",expiration_date)
    

    epoch = milliseconds_since_epoch(expiration_date)
    
    print("expiration_date:",expiration_date,epoch)
    

    price_history = TDSession.get_price_history(symbol = symbol, period_type = "day",period = 1, end_date = epoch, frequency_type = "minute", frequency = 30,extended_hours = False)
#    price_history = TDSession.get_price_history(symbol = symbol, period_type = "day",period = 1, frequency_type = "minute", frequency = 30,extended_hours = False)
#    price_history = TDSession.get_price_history(symbol = 'SPX',extended_hours = False)
    
    if price_history is None:
        print("cannot get closing price for: ",symbol,expiration_date)
        return None
    
#    print('get_closing_price: ',price_history,symbol,epoch,expiration_date)
    
#    for c in price_history['candles']:
#        print(c['datetime'],pd.to_datetime(c['datetime'],unit = 'ms'))
        
    
    #print("get_closing_price",price_history)
    
    candles = price_history["candles"]
    
    print("candles len = ",len(candles))
    
    for c in candles:
        print(c)
    
    if len(candles):
        return candles[-1]["close"]
    
    return None  # probably no close yet today while in session
    

def r(float_val):
    return "%.2f" % float_val


In [ ]:
def get_transactions_as_dataframe(transactions):
    
    
    df_options = pd.DataFrame(columns = ['ExpirationDate','Symbol','OptionSymbol','TransactionDate','OrderId','OrderIdIndex','positionEffect','amount','description','Cost'])

    first_t = None
    

    for t in transactions:
#    print(t)
        ti = t['transactionItem']
        i = ti['instrument']
        if 'assetType' not in i.keys():
            continue
        if i['assetType'] == 'OPTION':
            
            if i['underlyingSymbol'] != 'AAPL':
                continue
                
            #print("get_transactions_as_dataframe Dict:",t)
                

            orderId = t['orderId'].split('.')
            if len(orderId) == 1: #no orderId index
                orderId.append('0')
                
            expiration,strike = extract_expiration_strike(i['symbol'])
            
            underlyingSymbol = i['underlyingSymbol']
            
            if underlyingSymbol == 'SPXW':  # may have to strip 'W' off of every symbol
                underlyingSymbol = 'SPX'
            
            df_option = ({'ExpirationDate' : dt.strptime(i['optionExpirationDate'].split('+')[0],'%Y-%m-%dT%H:%M:%S'),
                                            'ExpirationDateStr' : i['optionExpirationDate'],
                                            'Symbol' : underlyingSymbol,
                                            'OptionSymbol' : i['symbol'],
                                            'TransactionDate': dt.strptime(t['transactionDate'].split('+')[0],'%Y-%m-%dT%H:%M:%S'),
                                            'TransactionDateStr' : t['transactionDate'],
                                            'OrderId' : orderId[0],
                                            'OrderIdIndex' : int(orderId[1]),
                                            'positionEffect' : ti['positionEffect'],
                                            'amount' : ti['amount'] if ti['cost']<=0 else -ti['amount'],
                                            'description' :i['description'],
                                            'strike' : strike,
                                            'putCall' : i['putCall'],
                                            'Cost': ti['cost']})
            
            #print("get_transactions df:",df_option['amount'])
            
            df_options = df_options.append(df_option,ignore_index=True)
            
            date_type_strike = re.split('_',i['symbol'])
            date_strike = re.split(r'[P,C]',date_type_strike[1])
            
            #print(i['underlyingSymbol {} dt.strptime {} i[optionExpirationDate] {}'],expiration,dt.strptime(i['optionExpirationDate'].split('+')[0],'%Y-%m-%dT%H:%M:%S'),i['optionExpirationDate'])
            exp1 = expiration
            #print(exp1)
            exp2 = dt.strptime(i['optionExpirationDate'].split('+')[0],'%Y-%m-%dT%H:%M:%S')
            #print(exp2)
            exp3 = i['optionExpirationDate']
            #print(exp3)
            #print('underlyingSymbol {} expiration {} dt.strptime {} i[optionExpirationDate] {}'.format(i['underlyingSymbol'],exp1,exp2,exp3))

#            print(t['transactionDate'],i['underlyingSymbol'],ti['instruction'],ti['positionEffect'],ti['amount'],'[',i['description'],']',ti['price'],ti['cost'])
#        print(i)
#        print()
    if i['assetType'] == 'EQUITY':
#        print(t['transactionDate'],i['symbol'],ti['instruction'],ti['amount'],ti['price'],ti['cost'])
        pass

    return df_options


In [ ]:
def calc_expiration_cost(row,closing_price):
    cost = 0
    if row['putCall'] == 'PUT':
        if closing_price < row['strike']:
            cost_option = (row['strike'] - closing_price) * 100 * row['amount']
            cost += cost_option
            #print("!!!!!!put held cost {} (strike: {} - closing_price: {}) * 100 * amount: () ".format(r(cost_option),row['strike'],closing_price,row['amount']))
                        
    if row['putCall'] == 'CALL':
        if closing_price > row['strike']:
            cost_option = (closing_price - row['strike']) * 100 * row['amount']
            cost += cost_option
            #print("!!!!!!call held cost {} (closing_price: {} - strike: {}) * 100 * amount: {} ".format(r(cost_option),closing_price,row['strike'],row['amount']))
            
    #print('expiration_cost: ',cost)

    return cost


In [ ]:

df_options = get_transactions_as_dataframe(transactions)

#print("df_options",df_options)

start_date = datetime.datetime(2020,4,21)

df_options.drop(df_options[df_options['TransactionDate'] < start_date].index,inplace=True)

df_options = df_options.sort_values(by='TransactionDate')  #sort to get in ascending order
        
        
#df_options_group = df_options.groupby(['ExpirationDate','Symbol','OrderId'])
df_options_group = df_options.groupby(['ExpirationDate','Symbol'])


cost_global = 0
cost_held_global = 0

for group_name, group in df_options_group:
    
    print("group expiration date:",group_name)
    
    
    group_expiration_date = group_name[0]
    group_symbol = group_name[1]
    
    if group_expiration_date >= datetime.datetime.now():
        continue  # next group if future expiration
    if group_expiration_date < date(2020,4,1):
        continue
        
        
    closing_price = get_closing_price(group_expiration_date,group_symbol)
    
    if closing_price is None:
        print("No closing price for: ",group_symbol, "is market in session?")
        continue

    cost = 0
    cost_held = 0
    print()
    print(group_expiration_date,group_symbol)
#    print(group)
    print()
    print()
    
    
    orderIdGroup = group.groupby(['OrderId'])
        
    group_cost = 0
    group_cost_held = 0
    
    
    for subgroup_name, subgroup in orderIdGroup:
        
        print("*************************************************************************************************",len(subgroup))

        print()
        
        subgroup_positions = {}

        cost = 0
        cost_held = 0
        for row_index, row in subgroup.iterrows():
            
#            print("row expiration str: ", row['ExpirationDateStr'])
#            print("row transaction str: ", row['TransactionDateStr'])

#            print("row: ",row)
            print("{} {} {} {}() {} [{}] {}".format(" " * 5,row['OrderIdIndex'],row['TransactionDate'],row['strike'],row['amount'],row['description'], row['Cost']))
            
            
            if row['description'] in subgroup_positions.keys():
                subgroup_positions[row['description']]['amount'] += row['amount']
            else:
                subgroup_positions[row['description']] = row
                
            if row['positionEffect'] == 'OPENING' and len(subgroup) > 1:  # only opening and not single puts, calls. 
                #print("OPENING")

                cost += row['Cost']
                cost_held += row['Cost']  #the premium paid or received
        
                expiration_cost = calc_expiration_cost(row,closing_price)
                cost_held += expiration_cost


    
            if row['positionEffect'] == 'CLOSING' and len(subgroup) > 1:  # only closing and not single puts, calls.
                print("CLOSING****************************************")
                cost += row['Cost']
                
                
        #print("\nchecking expiration positions\n")
                
        for key in subgroup_positions:
            row = subgroup_positions[key]
                
            expiration_cost = calc_expiration_cost(row,closing_price)
            cost += expiration_cost
                



                
        print(" " * 80,r(cost),r(cost_held))

        group_cost += cost
        group_cost_held += cost_held
        
    print(" " * 100,r(group_cost),r(group_cost_held))
    
    cost_global += group_cost
    cost_held_global += group_cost_held
    
    print(" " * 120, r(cost_global),r(cost_held_global))
        
    print("closing price:",closing_price)
    
            
    



In [ ]:
rtransactions = transactions.copy()
rtransactions.reverse()
cost = 0
for t in rtransactions:
    ti = t['transactionItem']

    i = ti['instrument']
    
    if 'assetType' in i.keys():
        
        if i['assetType'] == 'OPTION':
            cost += ti['cost']
            print("{} {:<6} {:6.0f}c {:10.2f} {:10.2f}".format(t['transactionDate'],i['underlyingSymbol'],ti['amount'],ti['cost'],cost))
        
        if i['assetType'] == 'EQUITY':
            cost += ti['cost']
            print("{} {:<6} {:6.0f}s {:10.2f} {:10.2f}".format(t['transactionDate'],i['symbol'],ti['amount'],ti['cost'],cost))



In [ ]:
type(transactions)

In [ ]:
from td.option_chain import *

In [ ]:
oc = OptionChain(symbol = "SPY")

In [ ]:
type(oc)

In [ ]:
oc._get_query_parameters()

In [ ]:
re.split('c','abc;def')

In [ ]:
date_type_strike

In [ ]:
df = pd.DataFrame([[1.1, 1.1, 1.1, 2.6, 2.5, 3.4,2.6,2.6,3.4,3.4,2.6,1.1,1.1,3.3], list('AAABBBBABCBDDD'), [1.1, 1.7, 2.5, 2.6, 3.3, 3.8,4.0,4.2,4.3,4.5,4.6,4.7,4.7,4.8], ['x/y/z','x/y','x/y/z/n','x/u','x','x/u/v','x/y/z','x','x/u/v/b','-','x/y','x/y/z','x','x/u/v/w'],['1','3','3','2','4','2','5','3','6','3','5','1','1','1']]).T
df.columns = ['col1','col2','col3','col4','col5']

In [ ]:
df


In [ ]:
d =df.groupby(['col5','col2'])

In [ ]:
d

In [ ]:
"abc".split('.')

In [ ]:
import time
dt = '2020-05-15T05:00:00+0000'.split('+')[0]

In [ ]:
t = datetime.strptime(dt,'%Y-%m-%dT%I:%M:%S')

In [ ]:
datetime.strptime('2020-05-15T05:00:00+0000','%Y-%m-%dT%I:%M:%S+0000')

In [ ]:
time.strftime("%Y-%m-%d",t)

In [ ]:
"x" * 3

In [ ]:
dt.now()

In [ ]:


print("epoch",group_name[0],milliseconds_since_epoch(dt.now()))
    
epoch = milliseconds_since_epoch(dt.now())
    
price_history = TDSession.get_price_history(symbol = "TSLA", period_type = "day", period=1, start_date = epoch)
    
    
    
print(len(price_history['candles']))


In [ ]:
d = dt.now()
milliseconds_since_epoch(d)

In [ ]:

symbol = 'TSLA'

date = dt(year = 2021,month = 1,day= 20,hour = 6,minute=30) #pst
date = dt(year = 2020,month = 4,day= 24,hour = 6,minute=30)

#Wednesday, January 20, 2021 2:30:00 PM
#expiration_date: 2020-04-24 06:30:00 1587709800000
print(date)

epoch = milliseconds_since_epoch(date)
d = dt.fromtimestamp(epoch/1000)
print('epoch:',epoch,d)




#price_history = TDSession.get_price_history(symbol = 'TSLA', period_type = "day", period = 1,end_date = 1611153000000, frequency_type = "minute", frequency = 30,extended_hours = False)
price_history = TDSession.get_price_history(symbol = 'AAPL', period_type = "day", period = 1,end_date = epoch, frequency_type = "minute", frequency = 30,extended_hours = False)


In [ ]:
len(price_history['candles'])

In [ ]:
price_history


In [ ]:
date = dt(year = 2020,month = 4,day= 23,hour = 6,minute=30)

epoch = milliseconds_since_epoch(date)

print(epoch)


In [ ]:
last = 0
for c in candles['candles']:
    print(c['datetime']- last)
    last = c['datetime']

In [10]:
from time import sleep
date = dt(year = 2021,month = 1,day= 20,hour = 6,minute=30) 
date = dt(year = 2020,month = 6,day= 18,hour = 6,minute=30) #Start of bad dates

for x in range(2000):
    epoch = milliseconds_since_epoch(date)
    sleep(.3)
    price_history = TDSession.get_price_history(symbol = 'AAPL', period_type = "day", period = 1,end_date = epoch, frequency_type = "minute", frequency = 30,extended_hours = False)
    if price_history:
        print(x,date,epoch,len(price_history['candles']))
    else:
        print(x,date,'NONE')
    date = date + pd.Timedelta(days=1)

0 2020-06-18 06:30:00 1592487000000 0
1 2020-06-19 06:30:00 1592573400000 0
2 2020-06-20 06:30:00 1592659800000 0
3 2020-06-21 06:30:00 1592746200000 0
4 2020-06-22 06:30:00 1592832600000 0
5 2020-06-23 06:30:00 1592919000000 0
6 2020-06-24 06:30:00 1593005400000 0
7 2020-06-25 06:30:00 1593091800000 0
8 2020-06-26 06:30:00 1593178200000 0
9 2020-06-27 06:30:00 1593264600000 0
10 2020-06-28 06:30:00 1593351000000 0
11 2020-06-29 06:30:00 1593437400000 0
12 2020-06-30 06:30:00 1593523800000 13
13 2020-07-01 06:30:00 1593610200000 13
14 2020-07-02 06:30:00 1593696600000 13
15 2020-07-03 06:30:00 1593783000000 13
16 2020-07-04 06:30:00 1593869400000 13
17 2020-07-05 06:30:00 1593955800000 13
18 2020-07-06 06:30:00 1594042200000 13
19 2020-07-07 06:30:00 1594128600000 13
20 2020-07-08 06:30:00 1594215000000 13
21 2020-07-09 06:30:00 1594301400000 13
22 2020-07-10 06:30:00 1594387800000 13
23 2020-07-11 06:30:00 1594474200000 13
24 2020-07-12 06:30:00 1594560600000 13
25 2020-07-13 06:30:00

KeyboardInterrupt: 

In [11]:
# Option Chain Example
opt_chain = {
    'symbol': 'MSFT',
    'contractType': 'CALL',
    'optionType': 'S',
    'fromDate': '2020-04-01',
    'afterDate': '2020-05-01',
    'strikeCount': 4,
    'includeQuotes': True,
    'range': 'ITM',
    'strategy': 'ANALYTICAL',
    'volatility': 29.0
}

# Get Option Chains
option_chains = TDSession.get_options_chain(option_chain=opt_chain)


In [12]:
option_chains

{'symbol': 'MSFT',
 'status': 'SUCCESS',
 'underlying': {'symbol': 'MSFT',
  'description': 'Microsoft Corporation - Common Stock',
  'change': -1.325,
  'percentChange': -0.5574,
  'close': 237.71,
  'quoteTime': 1616004876266,
  'tradeTime': 1616004875205,
  'bid': 236.36,
  'ask': 236.4,
  'last': 236.385,
  'mark': 236.385,
  'markChange': -1.325,
  'markPercentChange': -0.5574,
  'bidSize': 200,
  'askSize': 800,
  'highPrice': 236.59,
  'lowPrice': 233.23,
  'openPrice': 236.15,
  'totalVolume': 16369906,
  'exchangeName': 'NAS',
  'fiftyTwoWeekHigh': 246.13,
  'fiftyTwoWeekLow': 132.52,
  'delayed': False},
 'strategy': 'ANALYTICAL',
 'interval': 0.0,
 'isDelayed': False,
 'isIndex': False,
 'interestRate': 0.1,
 'underlyingPrice': 236.375,
 'volatility': 29.0,
 'daysToExpiration': -1.0,
 'numberOfContracts': 80,
 'putExpDateMap': {},
 'callExpDateMap': {'2021-03-19:2': {'232.5': [{'putCall': 'CALL',
     'symbol': 'MSFT_031921C232.5',
     'description': 'MSFT Mar 19 2021 232.5

In [16]:
items = json.loads('[{"id":1, "text": "item 1"}]')